<div style="background-color:#057BA3; padding:6px; color:#ffffff;">

## IUCAA Finesse Workshop 18-22nd Dec 2019 (Solved using Finesse 3)
</div>


# Plane Waves and Optical Interactions

$\renewcommand{\Re}{\operatorname{Re}}
\renewcommand{\Im}{\operatorname{Im}}$
Welcome to the Finesse Workshop! Our aim is to introduce you to modelling gravitational wave detectors with Finesse. By the end of the workshop we hope to have taught you how to model various aspects of a detector, such as producing your own quantum noise limited sensitivity curve for a gravitational wave detector.

These sessions together will cover the following material:
  * Fabry-Perot interferometers
  * Modulation and demodulation
  * Michelson interferometers
  * Locking and control
  * Noise couplings
  * Quantum noise

**Important reference material**:
  * Review article on interferometry: <a href="https://link.springer.com/article/10.1007/s41114-016-0002-8">Interferometer Techniques for Gravitational-Wave Detection</a>
  * Finesse main page: <a href="http://www.gwoptics.org/finesse">http://www.gwoptics.org/finesse</a>
  * <a href="http://www.gwoptics.org/finesse/reference/">Finesse online syntax reference</a>
  * <a href="http://www.gwoptics.org/finesse/reference/cheatsheet.php">Finesse online cheatsheet/FAQ</a>
  * <a href="http://kvasir.sr.bham.ac.uk/redmine/attachments/download/131/Finesse_2.0_manual.pdf">Finesse manual</a> (>200 pages pdf file)
  
Before we get started modelling optics in Finesse, it is important to understand the basic physics involved in an optics simulation. Here, we will briefly introduce the concepts that Finesse is based on. All of this information, and a lot more, is explained in detail in [
Interferometer Techniques for Gravitational-Wave Detection](https://link.springer.com/article/10.1007/s41114-016-0002-8).

*N.B. Amongst ourselves, we often refer to this paper (Interferometer Techniques for Gravitational-Wave Detection) as the "Living Review" (the journal it was published in). Just remember that if we mention the Living Review, we mean [
Interferometer Techniques for Gravitational-Wave Detection](https://link.springer.com/article/10.1007/s41114-016-0002-8).*

#### In this notebook:
  - Learn about the mathematical basis for almost all of the modelling we'll be doing
  - See how different components are represented in this basis

## Plane Waves

The choice of mathematical description used to describe light fields depends on the problems that we wish to solve. We start from the full description of the electric field for a par-axial (beam like) light field:

$$
\vec{E}(x, y, z, t) = E_0 \vec{e}_p \cos(\omega t - \vec{k}\vec{r} + \varphi) \cdot U(x, y, z).
$$

The notation used is as follows:

| Notation &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;              | Meaning |
:-----------------------|:---------|
| $\vec{a}$             | Vector  |
| $\vec{e}_p$           | Unit vector in direction of polarisation  |
| $\vec{E}(x, y, z, t)$ | Electric field vector as a function of space and time  |
| $E_0$                 | Electric field amplitude in units of $\mathrm{V/m}$ |
| $\omega$              | Angular frequency of oscillation, $\omega = 2\pi f$ |
| $\vec{k}$             | Vector in the direction of the beam axis, $k = |\vec{k}| = \omega / c$ |
| $\varphi$             | Phase offset in radians |
| $U(x, y, z)$          | Function descripting the shape of the beam |

This is quite a complex equation. For the purpose of modelling interferometers for gravitational wave detection, we can make some assumptions to simplify it:

1. We can ignore polarisation if there are no polarising elements present:

$$\vec{e}_p \rightarrow 1, \vec{E} \rightarrow E$$

2. We align our coordinate system with the beam axis:

$$\vec{K}\cdot\vec{r} = k \cdot z \rightarrow \cos(\omega t - \vec{k}\vec{r} + \varphi) = \cos(\omega t - kz + \varphi)$$

3. For now, we ignore the shape of the beam - this is called the "plane wave" model:

$$U(x, y, z) = 1$$

Note that this last point implies that the beam has an infinitely large cross-section, as it is now completely independent of $x$ and $y$. This is clearly unrealistic, so we don't model plane waves in practice. However, the equations for plane waves are useful as a model for laser beams, with the power on the axis of the plane wave representing the power in the beam, so we model just this on-axis part of the plane wave.

All of these points together give us a simplified equation for the beam,

$$ E = E_0 \cos(\omega t - k z + \varphi). $$

### Detecting beams

When we detect a beam, we use a component called a photodiode, which produces an electrical signal proportional to the **power** of the beam incident upon it (not the electric field!). The power at any one point $z_0$ is the cross-sectional integral of the beam's intensity,

$$P = \int\int I(x, y, z_0)\,\mathrm{d}x\mathrm{d}y,$$

where $I = c\varepsilon_0 E^2$. Setting $z_0 = 0$ to the location of our photodiode for convenience,

$$
\begin{align}
P &= c\varepsilon_0 E_0^2 \cos^2(\omega t)\\
&= \frac{c\varepsilon_0}{2} E_0^2 \left(1 + \cos(2 \omega t)\right)\\
&= \frac{c\varepsilon_0}{2} E_0^2
\end{align}
$$

You may have noticed that the $\cos(2\omega t)$ term dissapeared at the end. This is due to the timescales involved; in gavitational wave detectors, $\omega$ is on the order of $10^{15}\,$Hz. This is far too fast for any photodiode to detect, so we see the power of the field as constant.

### Complex Notation
Next, we introduce a complex notation for the field. We rewrite the real field as

$$E' = E_0'\cos(\omega t - k z + \varphi),$$
and redefine $E$ as
$$E = E_0 e^{i(\omega t - k z)}$$
with
$$E_0 = E_0'e^{i\varphi},$$

allowing us to simplify various operations. For example, a change in the phase of the field by $\phi$ can now be represented by a multiplication of $E$ with $e^{i\phi}$ (This is especially important for the method Finesse uses to model interferometers).

The electric field is now the real part of this complex field: $E' = \Re\{E\}$. It is important to remember that $E$ is **not** the actual field (complex numbers never describe physical quantities), merely a convenience for us to do calculations with. Finally, we adjust the units of $E_0$ to be $\sqrt{W}$ by absorbing a constant factor of $\sqrt{c\varepsilon_0 / 2}$ into our definition, so that the power in the field is simply given by $E_0 E_0^* = |E_0|^2$. We therefore have:

$$E = E_0 e^{i(\omega t - k z)}$$
$$P = |E_0|^2$$

This is the basis that we will work in for the rest of the workshop.

## Optical Interactions With Components

There are only 5 main components needed to understand how a basic interferometer works. Lasers act simply as a source of the electric field, and we have already seen how photodiodes detect the power in the beam at a specific point. We'll now take a look at the other three: mirrors, spaces, and beamsplitters.

### Mirror

<img src="assets/01-mirror_3d.svg" style="width: 300px; margin: auto; display: block;"/>

The image above shows laser light incident on a mirror, with some light transmitted and some reflected. For the purposes of illustration, we normally draw a simplified representation of the above mirror, with the fields labeled similarly to the below image.

<img src="assets/01-mirror.svg" style="width: 200px; margin: auto; display: block;"/>

For now, we will not consider the thickness of the mirror, and treat it as an infinitely thin plane. We can write the incident field $E_\mathrm{in}$ as

$$E_\mathrm{in} = E_0 e^{i\omega t},$$

taking $z=0$ to be the mirror's position for convenience. Consider what happens to this field upon reflection or transmission through the mirror. The $e^{i\omega t}$ term cannot change, as the field must be continuous at all points. The change must therefore be solely in the amplitude and phase of the field, $E_0$:

$$
\begin{align}
E_r &= r E_\mathrm{in}\\
E_t &= it E_\mathrm{in}
\end{align}
$$

where $r$ is the **amplitude** reflectivity of the mirror, and $t$ is its amplitude transmissivity. We call these equations the **coupling equations** for the mirror. The $i$ on transmission represents a $90^\circ$ phase change, which arises from energy conservation (for a full explanation of this, see the Living Review). Conversely, we can use the mirror's **power** reflectivity $R = r^2$ and transmissivity $T = t^2$. Real optics also have some amount of loss due to e.g. absorbing some of the laser light. For now, we will ignore any losses, meaning that $R + T = 1$ due to energy conservation.

*N.B. We've made many simplifications to reach this representation of a mirror. For the purposes of modelling the interferometers used in gravitational wave detectors, experience tells us that these simplifications are justified, but it is useful to remember that they are there for the situations where they do not hold.*

<div style="background-color:#cceffc;padding:10px;">

### Task 1: Mirror interaction

Given an input field $E_\mathrm{in}$ with a power of 1W incident on a mirror with reflectivity $r = 0.8$, calculate the power of:
1. The reflected field, $E_r$
2. The transmitted field, $E_t$ 
</div>

<div style="background-color:#c5e1a5;padding:10px;">
$P_{input} = 1 W $ <br>
$r = 0.8$<br>
$t=0.2$<br>
$R = r^{2}=0.64$<br>
$T = 1 - 0.64  = 0.36 $<br>
$P_{reflected} = R P_{input} = 0.64 W$<br>
$P_{transmitted} = T P_{input} =0.36 W$
</div>

In an interferometer, many mirrors will have light incident on both sides, not just one, so our picture above is incomplete. When multiple fields overlap, they simply add together. This allows us to write the full coupling equations for our mirror:

<img src="assets/01-mirror_full.svg" style="width: 200px; margin: auto; display: block;"/>

$$
\begin{align}
a_2 &= r a_3 + it a_1\\
a_4 &= r a_1 + it a_3
\end{align}
$$

There's a slight change in notation here, as we've renamed our various fields as $a_n$. They're just the same complex $E$ fields as before however. This is the notation used from hereon, and also in the Living Review.

### Space

<img src="assets/01-space.svg" style="width: 300px; margin: auto; display: block;"/>

Propagation through free space causes a phase change in the light proportional to the length of the space:

$$
\begin{align}
a_2 &= a_1 e^{-i k L}\\
a_4 &= a_3 e^{-i k L}
\end{align}
$$

Remember that $k$ is the wavenumber of the light, $\omega / c$, so this phase change depends on the light's frequency.

### Beamsplitter

<img src="assets/01-beamsplitter.svg" style="width: 200px; margin: auto; display: block;"/>

Beamsplitters can be considered as rotated mirrors; the only difference for now is the number of separate fields present. The couplings are as you might expect:

$$
\begin{align}
a_2 &= r a_1 + it a_7\\
a_4 &= r a_7 + it a_1\\
a_6 &= r a_5 + it a_3\\
a_8 &= r a_3 + it a_5\\
\end{align}
$$

## Aside: Coupling Matrices

As an alternative to the sets of equations given above, we can write the field couplings at a component as a matrix. For a mirror, the coupling matrix would be:

$$
\begin{pmatrix}
a_2\\
a_4
\end{pmatrix}
=
\begin{pmatrix}
it & r\\
r & it
\end{pmatrix}
\begin{pmatrix}
a_1\\
a_3
\end{pmatrix}
$$

and for a space:

$$
\begin{pmatrix}
a_2\\
a_4
\end{pmatrix}
=
\begin{pmatrix}
e^{-i k L} & 0\\
0 & e^{-i k L}
\end{pmatrix}
\begin{pmatrix}
a_1\\
a_3
\end{pmatrix}
$$

This is especially helpful when it comes to modelling these components on a computer, as many highly-efficient algorithms have been developed specifically for solving matrix problems like these. It also highlights one of the key advantages of the complex notation you've just been introduced to; if phase changes couldn't be represented by a simple multiplication, this matrix formulation would not work, and numerical modelling of detectors would be much more complicated.

# Fabry-Perot Cavities

#### In this notebook:
  - Calculate the response curve of a Fabry-Perot Cavity
  - Learn about the uses and features of this component

We now have formulas for the couplings of each of the components necessary to describe an interferometer. To begin with, let's consider one of the simplest interferometers possible; The Fabry-Perot cavity. The image below shows such a cavity, made up of two mirrors with **amplitude** reflectivities $r_1, r_2$ and transmissivities $t_1, t_2$.

<img src="assets/02-cavity.svg" style="width: 400px; margin: auto; display: block;"/>

<div style="background-color:#cceffc;padding:10px;">

### Task 1: Cavity transmission

Using the coupling equations given in the previous notebook, calculate the power in the field transmitted through the cavity above, $P_\mathrm{trans}$, in terms of the input power $P_0$. Can you see how a cavity could be used to detect gravitational waves?
    
**Hint:** Try calculating each field present, starting with the input: $a_1 \rightarrow a_2 \rightarrow a_2'$ etc.
</div>

<div style="background-color:#c5e1a5;padding:10px;">
We start with finding the relations between fields at different points. <br>
$a_{2} = it_{1}a_{1}+r_{1}a_{4}'$<br>
$a_{2}'=a_{2}e^{-ikl}$<br><br>
Using the first relation, we get<br>
$a_{2}'=it_{1}a_{1}e^{-ikl} + r_{1}a_{4}'e^{-ikl}$<br><br>
Also, $a_{4}'=r_{2}a_{2}'$<br>
$a_{2}'=it_{1}a_{1}e^{-ikl}+r_{1}r_{2}a_{2}'e^{-2ikl}$<br>
$a_{2}'=\frac{it_{1}a_{1}e^{-ikl}}{1-r_{1}r_{2}e^{-2ikl}}$<br>
$a_{3}=it_{2}a_{2}'$<br><br><br>
To find the Power, we evaluate the square of the modulus of these complex fields.<br><br>
$|a_{2}'|=\frac{a_{1}t_{1}}{\sqrt{1+r_{1}^{2}r_{2}^{2}-2r_{1}r_{2}cos(2kl)}}$<br>
$|a_{3}|=t_{2}|a_{2}'|$<br>
$|a_{3}|=\frac{t_{2}a_{1}t_{1}}{\sqrt{1+r_{1}^{2}r_{2}^{2}-2r_{1}r_{2}cos(2kl)}}$<br>
$|a_{0}|^{2}=P_{0}$<br>
$P_{3}=P_{trans}=\frac{T_{2}P_{0}T_{1}}{1+R_{1}R_{2}-2\sqrt{R_{1}R_{2}}cos(2kl)}$<br>
$\frac{P_{trans}}{P_{0}}=\frac{T_{2}T_{1}}{1+R_{1}R_{2}-2\sqrt{R_{1}R_{2}}cos(2kl)}$<br><br>
The Power transmitted is maximum when the cosine term yields 1. In other words, when $2kl=2n\pi$.<br>
The corresponding frequencies are called the resonance frequencies and high power transmission can be obtained at regions close to these frequencies.<br>
$$f=\frac{nc}{2l}$$
</div>

## Cavity Features

Below, we've plotted how the transmitted power you've just derived varies with either the length of the cavity, $L$, or the frequency of the laser, $f$.

<img src="assets/02-cavity_response.svg" style="width: 500px; margin: auto; display: block;"/>

As $f$ and $L$ change, we see the transmitted power rise and fall. Note that the shape will also be the same for the circulating power inside the cavity—more on this later. We say that the cavity is "on resonance" when the circulating power is at a maximum. The key quantities of this plot that we are interested in are the difference between two resonances, known as the "Free Spectral Range" (FSR), and the "Full Width at Half Maximum" (FWHM) of each of the peaks. The "Finesse", $\mathcal{F}$, or quality factor of the cavity is given by the ratio:

$$
\mathcal{F} = \frac{\mathrm{FSR}}{\mathrm{FWHM}}
$$

We'll have a look at what the finesse means for the behaviour of the cavity later.

<div style="background-color:#cceffc;padding:10px;">

### Task 2: Resonances

1. Why is a change in $f$ or $L$ equivalent?
1. What is the condition on $f$ and $L$ for resonance in a cavity? (When is your equation for power maximum?)
1. What is the FSR of the cavity, in terms of the frequency $f$?
</div>

<div style="background-color:#c5e1a5;padding:10px;">
The change in $f$ and $l$ are equivalent for they have the same relationship with the power transmitted. k indirectly relates f to the power. 
$$k=\frac{2\pi f }{c}$$<br>
Thus, Changes in $f$ or $l$  in the cosine term, effect the Power in the same way.<br><br>
The conditions for the maximum power transmission:<br>
$$\frac{2\pi}{\lambda}=\frac{2\pi f}{c}=\frac{n\pi}{l}$$<br>
Equivalently, the resonant frequencies in the cavity are$$f=\frac{nc}{2l}$$<br>
Length of the cavity, $$l =\frac{n\lambda}{2}$$<br>
where n can be any positive integer.<br><br>
FSR of the cavity in terms of frequency can be figured out by evaluating the difference of two consecutive resonant frequencies. $$FSR_{f}=\frac{c}{2l}$$
</div>

From the plot above we can see that on resonance, the cavity transmits all of the input light from the laser, with the amount transmitted decreasing as we move away from resonance. This shows one of the uses for cavities: a **spectral filter**. If light at the resonance frequency of the cavity, $\omega_0$, is incident upon it, it will all pass through. Light at different frequencies will be partially reflected, wih more and more being reflected as we move away from resonance. This leaves us with a beam mostly made up of frequencies near the resonance frequency.

The spectral filtering property of a cavity can also be used for another purpose. By measuring the amount of light transmitted, we can clearly see any deviation of the cavity length or the laser frequency from resonance, as the measured power will decrease. In this way, a cavity can be used to **stabilise** a light source, by allowing us to measure the frequency very precisely.

While these are both useful features of cavities, they are not the main reason we use them in gravitational wave detectors. It's helpful to look first at the power of the field circulating *inside* the cavity, for example that of $a_4$ in our diagram:

$$
P_\mathrm{circ} = P_0 \frac{T_1 R_2}{1 + R_1 R_2 - r_1 r_2 \cos{2 k L}}
$$

You should notice that this looks very similar to your equation for the power transmitted through the cavity. In fact, all cavity fields have this same shape! The only difference is the factor $R_2$ instead of $T_2$ in the numerator, which will vary depending on where in the cavity you measure the field. This change leads to the most useful feature of cavities for GW detectors. Let's plot the power again, this time looking at the circulating power:

<img src="assets/02-circulating_power.svg" style="width: 500px; margin: auto; display: block;"></a>

You can see that on resonance, we have much more power circulating in the cavity than we put in! **This is the main reason we use cavities in GW interferometers.** Cavities allow us to achieve powers greater than we ever could with simply a laser. The most powerful sustained lasers we have today reach ~1 kW; the arm cavities in LIGO have 750 kW of laser power circulating in them!

### Cavity Finesse

We have already mentioned that the finesse of a cavity is its quality factor. This is clear if we plot low and high finesse cavities together:

<img src="assets/02-cavity_finesse.svg" style="width: 500px; margin: auto; display: block;"></a>

For the high finesse cavity, the power buildup at resonance is much greater than in the low finesse cavity, but off-resonance the buildup is much lower.

In general, the finesse of a cavity is a complicated equation, as it depends on the FWHM of the resonances (which is itself complicated). In interferometers for GW detection, however, we are often only interested in high-finesse cavities where $r_1$ and $r_2$ are close to 1. In this case, we can approximate:

$$
\begin{align}
\mathcal{F} &\approx \frac{\pi\sqrt{r_1 r_2}}{1 - r_1 r_2}\\
&\approx \frac{\pi}{1 - r_1 r_2}
\end{align}
$$

## Why use Finesse?

Hopefully by now you've seen that the mathematics behind deriving properties of just a simple two mirror setup can be very complicated, even with all the approximations we've made. For a full interferometer layout, there can be up to a hundred or so components involved, with multiple light frequencies and non-plane-wave beams; doing the calculations on paper is infeasible! Instead, we can use computers to do the work for us, by simulating the interferometer numerically. This is the purpose of Finesse, which you will be introduced to in the next session.